In [1]:
import math
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from ces_renewable_util import *
#from fun_utils_eq import *

In [2]:
## parameter values
theta = 4             # scopevec for comparative advantage
sigma = 1      # elasticity of demand for each individual manufactured good j at Home
sigmastar = 1  # elasticity of demand for each individual manufactured good j at Foreign
beta=1.892412
gamma=0.807998928
epsilonS = 0.5  #Homes's energy supply elasticity: beta/(1 - beta)
epsilonSstar = 0.5  #Foreign's energy supply elasticity: betastar/(1 - betastar)
epsilonSstar1 = 0.5
epsilonS1 = 0.5
epsilonSstar2 = 2
epsilonS2 = 2

In [3]:
# carbon content per unit of energy
h1 = 1
h2 = 0
# make sure that epsilonSvec and epsilonSstarvec have equal length
# if a country does not have that energy source, set proportion to 0
epsilonSvec = [(epsilonS1, 1, 0.9), (epsilonS2, h2, 0.1)]
epsilonSstarvec = [(epsilonSstar1, 1,0.85), (epsilonSstar2, h2, 0.15)]
# baseline
epsilonSvec = [(epsilonS1, h1, 1), (epsilonS2, h2, 0)]
epsilonSstarvec = [(epsilonSstar1, h1, 1), (epsilonSstar2, h2, 0)]
assert(sum(k for i,j,k in epsilonSvec) == 1)
assert(sum(k for i,j,k in epsilonSstarvec) == 1)

In [4]:
## scenario switch
case=3 # 2 means no trade in goods; 3 means trade in both energy and goods
logit = 0 # 1 means logit estimations of supply elasticity; 0 means fixed elasticities at 0.5

In [5]:
paralist = (theta, sigma, sigmastar, epsilonSvec, epsilonSstarvec, beta, gamma, logit)

In [6]:
class taxrow_eq:
    def __init__(self, pe, te, tb_mat, df, tax_scenario, varphi, paralist):
        self.df = df
        self.tax_scenario = tax_scenario
        self.varphi = varphi
        self.paralist = paralist
        self.prop = 1
        self.te = te
        self.pe = pe
        self.tb_mat = tb_mat

    ## computes equilibrium price and taxes. Also computes other equilibrium values (ie consumption, production, value of exports/imports)
    ## and stores them in self. 
    def opt_tax(self):
        tax_scenario = self.tax_scenario
        varphi = self.varphi
        df = self.df
        te = varphi
        self.region = df['region_scenario']
        pes = np.append([self.pe], np.arange(0.1, 2, 0.2))
        tbs = np.append([self.tb_mat[0]], np.arange(0,1.5,0.2))
        props = np.append([self.tb_mat[1]], np.arange(0,1.1,0.5))
        self.conv = 1
        if (tax_scenario['tax_sce'] == 'global'):
            tbs = [0]
        
        if tax_scenario['tax_sce'] != 'purete' and tax_scenario['tax_sce'] != 'EP_hybrid':
            for prop in props:
                for tb in tbs:
                    for pe in pes:
                        res = minimize(self.minuswelfare, [pe, tb, prop], bounds=[(0.01, np.inf), (0, np.inf), (0, 1)],
                               method='nelder-mead', args=(te), tol=0.000001, options={'maxfev': 100000})
                        if res.fun <= 0.0001:
                            opt_val = res.x
                            break
                    else:
                        continue
                    break
                else: 
                    continue
                break
            if res.fun > 0.0001:
                self.conv = 0
                print("did not converge, phi is", varphi, "init guess is", self.pe, self.tb_mat, 'region is', tax_scenario['tax_sce'], df['region_scenario'])
            opt_val = res.x
            #print(res.fun, df['region_scenario'], tax_scenario['tax_sce'], varphi)
            
        pes = np.append([self.pe], np.arange(0.1, 2, 0.5))
        props = np.append([self.tb_mat[1]], np.arange(0,2,0.5))
        if tax_scenario['tax_sce'] == 'EP_hybrid':
            for pe in pes:
                for tb in tbs:
                    for prop in props:
                        res = minimize(self.minuswelfare, [pe, tb, prop], bounds=[(0.01, np.inf), (0, np.inf), (0, np.inf)],
                               method='nelder-mead', args=(te), tol=0.000001, options={'maxfev': 100000})
                        if res.fun <= 0.0001:
                            opt_val = res.x
                            break
                    else:
                        continue
                    break
                else: 
                    continue
                break
            if res.fun > 0.0001:
                self.conv = 0
                print("did not converge, phi is", varphi, "init guess is", self.pe, self.tb_mat, 'region is', tax_scenario['tax_sce'], df['region_scenario'])
            opt_val = res.x
            
            
        ## not implemented yet
        if tax_scenario['tax_sce'] == 'Baseline':
            self.tb = 0
            self.prop = 1
            self.te = 0
        
        elif tax_scenario['tax_sce'] == 'global':
            self.tb = 0
            self.te = varphi

        elif tax_scenario['tax_sce'] == 'PC_hybrid':
            self.tb = opt_val[1]
            self.prop = opt_val[2]
            self.te = self.tb

        elif tax_scenario['tax_sce'] == 'EPC_hybrid':
            self.tb = opt_val[1]
            self.prop = opt_val[2]
            self.te = varphi

        elif tax_scenario['tax_sce'] == 'EP_hybrid':
            self.tb = opt_val[1]
            self.te = opt_val[2]
            self.prop = self.te #- self.tb

        elif tax_scenario['tax_sce'] == 'Unilateral' or tax_scenario['tax_sce'] == 'puretc' or tax_scenario[
            'tax_sce'] == 'puretp' or tax_scenario['tax_sce'] == 'EC_hybrid':
            self.tb = opt_val[1]
            self.prop = opt_val[2]

            if tax_scenario['tax_sce'] == 'puretc' or tax_scenario['tax_sce'] == 'puretp':
                self.te = self.tb

            elif tax_scenario['tax_sce'] == 'Unilateral' or tax_scenario['tax_sce'] == 'EC_hybrid':
                self.te = self.varphi

        elif tax_scenario['tax_sce'] == 'purete':
            res = minimize(self.minuswelfare, [self.pe, self.te], bounds=[(0.001, np.inf), (0, np.inf)], method='nelder-mead', args=(0),
                           tol=0.000001, options={'maxfev': 100000})
            
            if res.fun > 0.0001:
                self.conv = 0
                print("did not converge, phi is", varphi, "init guess is", self.pe, self.tb_mat, 'region is', tax_scenario['tax_sce'], df['region_scenario'])
            #print(res.fun)
            self.te = res.x[1]
            tb_mat = [0, 1]
            self.tb = tb_mat[0]
            self.prop = tb_mat[1]
        #print(tax_scenario['tax_sce'], res)

    def minuswelfare(self, p, te):
        pe = p[0]
        tb_mat = p[1:]
        varphi = self.varphi
        tax_scenario = self.tax_scenario
        paralist = self.paralist
        df = self.df

        if tax_scenario['tax_sce'] == 'purete':
            #te = p[1]
            #tb_mat = [0, 1]
            te = varphi

        obj = self.comp_obj(pe, te, tb_mat, varphi, tax_scenario, paralist, df)

        return obj

    ## compute the objective value, currently the objective is to minimize negative welfare
    ## also saves optimal results in self.
    def comp_obj(self, pe, te, tb_mat, varphi, tax_scenario, paralist, df):

        theta, sigma, sigmastar, epsilonSvec, epsilonSstarvec, beta, gamma, logit = paralist
        ## compute extraction tax, and jbar's
        te, tb_mat, jmbar_hat, jxbar_prime, j0_prime, jmbar_prime = comp_jbar(paralist, pe, te, varphi, tb_mat, tax_scenario, df)
        jvals = (j0_prime, jxbar_prime, jmbar_hat, jmbar_prime)

        # if te is too large, Home stop producing
        petbte = pe + tb_mat[0] - te
        if petbte < 0:
            petbte = 0

        Qe_prime, Qestar_prime, Qes, Qestars = compute_qe(pe, tb_mat,varphi, te, df, paralist)
        Qeworld_prime = Qe_prime + Qestar_prime

        Cey_prime, Cex1_prime, Cex2_prime, Cex_prime, Cem_prime, Ceystar_prime = comp_ce(pe, tb_mat, jvals, paralist, df, tax_scenario)
        consvals = (Cey_prime, Cex1_prime, Cex2_prime, Cex_prime, Cem_prime, Ceystar_prime)
        
        Ge_prime = Cey_prime + Cex_prime
        Gestar_prime = Ceystar_prime + Cem_prime
        Ce_prime = Cey_prime + Cem_prime
        Cestar_prime = Ceystar_prime + Cex_prime

        Vgy_prime, Vgm_prime, Vgx1_prime, Vgx2_prime, Vgx_prime, Vgystar_prime = comp_vg(pe, tb_mat, jvals, consvals, df, tax_scenario, paralist)
        
        vgvals = Vgy_prime, Vgm_prime, Vgx1_prime, Vgx2_prime, Vgx_prime, Vgystar_prime
        Vg, Vg_prime, Vgstar, Vgstar_prime = comp_vgfin(pe, tb_mat, consvals, vgvals, jvals, paralist, df, tax_scenario)

        subsidy_ratio = 1- ((1-jxbar_prime) * j0_prime / ((1-j0_prime) * jxbar_prime))**(1/theta)

        ## compute Ve values
        Ve_prime, Vestar_prime = comp_ve(pe, tb_mat, consvals, tax_scenario)

        Lg, Lg_prime, Lgstar, Lgstar_prime = comp_lg(pe, tb_mat, Ge_prime, Gestar_prime, consvals, paralist, df, tax_scenario)

        leakage1, leakage2, leakage3 = comp_leak(Qestar_prime, Gestar_prime, Cestar_prime, Qeworld_prime, df)

        lgvals = Lg, Lgstar, Lg_prime, Lgstar_prime
        vgfinvals = Vg, Vgstar, Vg_prime, Vgstar_prime
        delta_Le, delta_Lestar, delta_U, delta_Vg, delta_Vgstar = comp_delta(lgvals, vgfinvals, Qeworld_prime, df, jvals, pe, te, tb_mat, tax_scenario, varphi, paralist)

        chg_extraction, chg_production, chg_consumption, chg_Qeworld = comp_chg(df, Qestar_prime, Gestar_prime,
                                                                                Cestar_prime, Qeworld_prime)

        welfare = delta_U / Vg * 100
        welfare_noexternality = (delta_U + varphi * (Qeworld_prime - df['Qeworld'])) / Vg * 100
        
        self.results = assign_val(Ge_prime, Gestar_prime, Lg_prime, Lgstar_prime, Qe_prime, Qestar_prime, Qeworld_prime,
                                  Ve_prime, Vestar_prime,vgvals, vgfinvals, chg_Qeworld, chg_consumption, chg_extraction, chg_production, delta_Le, 
                                  delta_Lestar, leakage1, leakage2, leakage3, pe, subsidy_ratio, varphi, 
                                  welfare, welfare_noexternality, jvals, consvals, delta_Vg, delta_Vgstar, Ce_prime,Cestar_prime, Qes, Qestars)

        diff, diff1, diff2 = comp_diff(consvals, jvals, Qes, Qestars, Qe_prime, Qestar_prime, Qeworld_prime,Vgx2_prime, pe, tax_scenario, tb_mat, te, varphi, paralist, df)

        obj_val = abs(diff1) + abs(diff) + abs(diff2)
        return obj_val

    ## retrive the pandas series object containing equilibrium values (prices, taxes, consumption etc)
    def retrive(self):
        ret = self.results
        ret['tb'] = self.tb
        ret['prop'] = self.prop
        ret['te'] = self.te
        ret['region_scenario'] = self.region
        ret['conv'] = self.conv
        
        return ret

In [7]:
## import BAU values (seven regional scenarios in the order of US, EU, OECD, World, China, OECD plus China)
if case==2:
    df = pd.read_csv("../../output/BaselineCarbon_2015_noTradeinGoods.csv",index_col=['region_scenario','regionbase'],header='infer')
elif case==3:
    df = pd.read_csv("../../output/BaselineCarbon_2015.csv", index_col=['regionbase'],header='infer')
df['jxbar']=df['CeFH']/(df['CeFH'] + df['CeFF'])
df['jmbar']=df['CeHH']/(df['CeHH'] + df['CeHF'])

## choose which regional scenario to run (runs all if not executed)
#df=df.drop([2,3,4,5,6,7])  
df = df[df['region_scenario'] == 3]
#df = df[df['region_scenario'] != 4]
df

region_scenario       Qe   Qestar   Qeworld      CeHH  \
regionbase                                                              
OECD37 as Home                3  8.62549  23.6505  32.27599  11.29367   

                   CeHF     CeFH      CeFF        Ce    Cestar        Ge  \
regionbase                                                                 
OECD37 as Home  2.48754  0.91057  17.58421  13.78121  18.49478  12.20424   

                  Gestar   Ceworld   Geworld     jxbar     jmbar  
regionbase                                                        
OECD37 as Home  20.07175  32.27599  32.27599  0.049234  0.819498

In [8]:
tax_scenario= pd.DataFrame({'tax_sce': ['Unilateral','global','purete','puretc','puretp','EC_hybrid','EP_hybrid','PC_hybrid','EPC_hybrid'], 'Base':[0,1,1,1,1,1,1,1,1]},index=[1,2, 3, 4, 5, 6, 7, 8,9])
tax_scenario= pd.DataFrame({'tax_sce': ['Unilateral','purete','puretc','puretp','EC_hybrid','EP_hybrid','PC_hybrid','EPC_hybrid'], 'Base':[0,1,1,1,1,1,1,1]},index=[1,2, 3, 4, 5, 6, 7, 8])
tax_scenario= pd.DataFrame({'tax_sce': ['purete'], 'Base':[1]},index=[1])
#tax_scenario= pd.DataFrame({'tax_sce': ['Unilateral'], 'Base':[0]},index=[1])
#df = pd.DataFrame({'Qe': 4.4800448, 'Qestar': 27.795946, 'Qeworld': 32.27599, 'CeHH': 4.598403, 'CeHF': 1.196111,'CeFH': 0.42160103,'CeFF': 26.059874,'Ce': 5.7945137,'Cestar': 26.481474,'Ge': 5.0200038,'Gestar': 27.255985,'Ceworld': 32.27599,'Geworld': 32.27599,'jxbar': 0.015920602214279298,'jmbar':  0.7935787194577492}, index = [1])

In [9]:
def temp(tax_scenario,theta, sigma, sigmastar, epsilonSvec, epsilonSstarvec, beta, gamma, logit):
    paralist = (theta, sigma, sigmastar, epsilonSvec, epsilonSstarvec, beta, gamma, logit)
    tb_mat = [0, 1];  #initial value of  border adjustment and proportion of it (prop is mainly used for PC hybrid)
    # use for quick test: varphilist = [2] or varphilist = np.arange (1.7,2.5,0.1)
    #varphilist = np.arange(0,3,0.1)
    varphilist = np.arange (0,10,0.1)
    varphilist = [1]
    output=[]
    prevtb = [1,1,1,1]
    for varphi in varphilist:    
        tax_df=df.apply(mintemp, axis=1, raw=False, args=(prevtb, varphi, tax_scenario, paralist))
        prevtb = tax_df[['region_scenario','tb', 'pe', 'te', 'prop']]
        output.append(tax_df)
        print(varphi)
    output = pd.concat(output, axis=0, join='outer',  ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False,copy=True)
    output.reset_index(level=0, inplace=True)
    output = output.sort_values(by=['region_scenario','varphi'])
    if tax_scenario['tax_sce']=='purete' or tax_scenario['tax_sce']=='EP_hybrid':
        output.te[output.Qe_prime==0]=output.pe+output.tb
    print(tax_scenario['tax_sce'])
    return output

In [10]:
def mintemp(df, prevtb, varphi, tax_scenario, paralist):
    pe = 1
    te = 0.5
    tb_mat = [0,0.5]
    #print(prevtb['region_scenario'])
    
    if (type(prevtb) != list):
        curr_region = prevtb[prevtb['region_scenario'] == df['region_scenario']]
        vals = curr_region.values
        #print(vals)
        tb_mat = [vals[0][1], vals[0][4]]
        pe = vals[0][2]
        te = vals[0][3]
    tax_temp = taxrow_eq(pe, te, tb_mat, df, tax_scenario, varphi, paralist)
    tax_temp.opt_tax()
    ret = tax_temp.retrive()
    #print(ret)
    return ret

In [11]:
temp_df = tax_scenario.apply(temp, axis=1, args=(theta, sigma, sigmastar, epsilonSvec, epsilonSstarvec, beta, gamma, logit))
#temp_df.head()

1
purete


In [12]:
output_list=[]
for i in range(1,len(tax_scenario)+1):
    output_list.append(temp_df.loc[i])
Outcomes = pd.concat(output_list, axis=0, join='outer', ignore_index=False, keys=tax_scenario['tax_sce'], levels=None, verify_integrity=False,copy=True)
Outcomes.reset_index(level=0, inplace=True)
Outcomes

tax_sce      regionbase  varphi        pe   tb  prop        te  jxbar_prime  \
0  purete  OECD37 as Home     1.0  1.029948  0.0   1.0  0.293369     0.049234   

   jmbar_prime  j0_prime  ...  delta_Vg  delta_Vgstar    welfare  \
0     0.819498  0.042653  ... -0.406663     -0.545754  20.239014   

   welfare_noexternality  Qe1_prime  Qe1star_prime  Qe2_prime  Qe2star_prime  \
0              16.571608   4.904538      24.002033        0.0            0.0   

   region_scenario  conv  
0              3.0   1.0  

[1 rows x 53 columns]

In [13]:
Outcomes[['pe','tb', 'Cey_prime', 'Cex_prime', 'Cem_prime', 'Ceystar_prime', 'Vgx_prime', 'Vgm_prime']]

pe   tb  Cey_prime  Cex_prime  Cem_prime  Ceystar_prime  Vgx_prime  \
0  1.029948  0.0   8.534364   0.884093   2.415209      17.072907   6.070467   

   Vgm_prime  
0    16.5836

In [14]:
#outcome_t = Outcomes[Outcomes['regionbase'] == 'US as Home']
Outcomes[['pe','tb', 'Qe_prime', 'Qestar_prime', 'Ce_prime', 'Cestar_prime','Lg_prime','Lgstar_prime', 'Vg_prime', 'Vgstar_prime', 'Vgx2_prime', 'delta_Le','delta_Lestar','welfare','welfare_noexternality']]

pe   tb  Qe_prime  Qestar_prime   Ce_prime  Cestar_prime   Lg_prime  \
0  1.029948  0.0  4.904538     24.002033  10.949573        17.957  54.969629   

   Lgstar_prime   Vg_prime  Vgstar_prime  Vgx2_prime  delta_Le  delta_Lestar  \
0    113.739917  75.183285     64.670151    0.811401  -2.34659      0.356783   

     welfare  welfare_noexternality  
0  20.239014              16.571608

In [15]:
#Outcomes.to_csv('../../output_renewable/all0_90_85.csv_higheps'.format(theta, case), header=True)